In [ ]:
%%configure -f
{
    "conf":{"spark.driver.memory":"3g","spark.executor.memory":"4g","spark.executor.cores":2,"spark.executor.instances":8,"spark.jars":"/spark-sql-perf-assembly-0.7.1-SNAPSHOT.jar"}
}

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

resultLocation = "abfs://sparkkc@linjzhukcadls.dfs.core.windows.net/result" 
resultTable = spark.read.json(resultLocation)
resultTable.createOrReplaceTempView("sqlPerformance")

ds = spark.sql("""select tsVersion,substring(result.name,1,length(result.name)-5) as name,(result.parsingTime+result.analysisTime+result.optimizationTime+result.planningTime+result.executionTime)/1000.0 as runtime from (
select concat_ws('-',timestamp,configuration.sparkVersion) as tsVersion, explode(results) as result from sqlPerformance where timestamp in (1692255307792,1692170386179,1692274972828)
    )""")
pdf = ds.toPandas()


plt.figure(figsize=(30,20))
for label, grp in pdf.groupby('tsVersion'):
    print(label)
    print(grp)
    plt.plot(grp["name"], grp["runtime"], label = label)
   

plt.xticks(rotation=45)
plt.legend()
plt.show()

In [ ]:
%%sql
select timestamp,result.failure.* from (
select timestamp, explode(results) as result from sqlPerformance
    )

In [ ]:
%matplot plt